Analyse de données sur le jeu "League of Legends"

Dans ce projet, nous allons explorer et analyser un ensemble de données liées au jeu vidéo populaire "League of Legends". L'objectif principal de cette analyse est de découvrir des informations intéressantes et potentiellement utiles à partir des données de jeu. Plus précisément, nous cherchons à comprendre comment certains facteurs, tels que l'avantage financier et positionnel, l'élimination de monstres spécifiques, et les champions choisis, peuvent influencer l'issue d'un match.

Les données que nous utilisons proviennent de plusieurs fichiers CSV et JSON, qui contiennent des informations détaillées sur une série de matchs, y compris des statistiques à 15 minutes et à la fin du match, ainsi que des informations sur les champions utilisés.

Notre analyse comprendra les étapes suivantes:

    Extraction des informations pertinentes de nos fichiers de données et organisation de ces informations dans des DataFrames pandas pour faciliter la manipulation et l'analyse.

    Création de nouvelles variables basées sur les données existantes, telles que l'avantage financier et positionnel d'une équipe et le nombre de monstres spécifiques tués.

    Analyse des relations entre ces variables et l'issue des matchs, pour déterminer quels facteurs sont les plus fortement associés à la victoire.

    Exportation de nos résultats sous forme de fichiers JSON pour une utilisation et une analyse ultérieures.

A travers ce projet, nous espérons non seulement répondre à nos questions de recherche, mais aussi démontrer la puissance de l'analyse de données dans le contexte des jeux vidéo, un domaine qui est de plus en plus reconnu pour sa richesse en données intéressantes et exploitables.

1. Ouvrir le fichier de « match_15m.csv » et stocker dans un nouveau DataFrame les informations suivantes pour
chaque match :
o blue_advantage_gold : est-ce que l’équipe bleue est en avantage financier (plus d’argent que l’équipe
rouge)
o blue_advantage_buildings : est-ce que l’équipe bleue est en avantage de position (plus de bâtiment
de l’équipe adverse détruit)
o blue_drake : nombre de dragons tués par l’équipe bleue
o red_drake : nombre de dragons tués par l’équipe rouge
o blue_herald : est-ce que l’équipe bleue a tué le Rift Herald
o red_herald : est-ce que l’équipe rouge a tué le Rift Herald
o blue_win : est-ce que l’équipe bleue a gagné
2. Ce DataFrame devra être exporté au format JSON avec le nom « df_ex1.json »

In [20]:
import pandas as pd

# Lire le fichier CSV
match_data = pd.read_csv("data/match_15m.csv")

# Exercice 1
match_data["blue_advantage_gold"] = match_data["blue_gold"] > match_data["red_gold"]
match_data["blue_advantage_buildings"] = (match_data["blue_tower"] + match_data["blue_inhibitor"]) > (match_data["red_tower"] + match_data["red_inhibitor"])

# Compter le nombre de dragons tués par l'équipe
match_data["blue_drake"] = match_data["blue_monster"].apply(lambda x: x.count("DRAGON"))
match_data["red_drake"] = match_data["red_monster"].apply(lambda x: x.count("DRAGON"))

# Vérifier si l'équipe a tué le Rift Herald
match_data["blue_herald"] = match_data["blue_monster"].apply(lambda x: "HERALD" in x)
match_data["red_herald"] = match_data["red_monster"].apply(lambda x: "HERALD" in x)

# Sélectionner les colonnes pertinentes
df_ex1 = match_data[['matchId', 'blue_advantage_gold', 'blue_advantage_buildings', 'blue_drake', 'red_drake', 'blue_herald', 'red_herald', 'blue_win']]

# Exporter au format JSON
df_ex1.to_json("df_ex1.json", orient='records')




3. A partir de ce fichier, créer un nouveau DataFrame avec l’information suivante :
o Créer une répartition des écarts d’or de l’équipe bleue (advantage_blue_category) comme suit :
▪ Avantage fort : [2500, 5000]
▪ Avantage faible : [500, 2500]
▪ Pas d’avantage : [−500, 500]
▪ Désavantage faible : [−2500, −500]
▪ Désavantage fort : [−5000, −2500]
o Pour chaque écart, avoir une colonne « chance_win » indiquant le pourcentage de parties gagnées
o Pour chaque écart, avoir une colonne « blue_structure » indiquant la moyenne du nombre de
structures détruites dans l’équipe bleue
o Pour chaque écart, avoir une colonne « red_structure » indiquant la moyenne du nombre de
structures détruites dans l’équipe rouge
o Pour chaque écart, avoir une colonne « blue_monster » indiquant la moyenne du nombre de monstres
tués (dragons et herald combinés) pour l’équipe bleue
o Pour chaque écart, avoir une colonne « red_monster » indiquant la moyenne du nombre de monstres
tués (dragons et herald combinés) pour l’équipe rouge
o Ce DataFrame devra être exporté au format JSON avec le nom « df_ex3.json ».

In [18]:


# Exercice 3
match_data['gold_diff'] = match_data['blue_gold'] - match_data['red_gold']

bins = [-5000, -2500, -500, 500, 2500, 5000]
labels = ['Désavantage fort', 'Désavantage faible', 'Pas d’avantage', 'Avantage faible', 'Avantage fort']
match_data['advantage_blue_category'] = pd.cut(match_data['gold_diff'], bins=bins, labels=labels)

# Calculer le nombre total de monstres tués par chaque équipe
match_data['blue_total_monsters'] = match_data['blue_drake'] + match_data['blue_herald']
match_data['red_total_monsters'] = match_data['red_drake'] + match_data['red_herald']

grouped_data = match_data.groupby('advantage_blue_category').agg(
    chance_win=('blue_win', 'mean'),
    blue_structure=('blue_tower', 'mean'),
    red_structure=('red_tower', 'mean'),
    blue_monster=('blue_total_monsters', 'mean'),
    red_monster=('red_total_monsters', 'mean')
)

grouped_data['chance_win'] *= 100

grouped_data.to_json("df_ex3.json")





4.Que pouvez-vous déduire de ces données ?

Voici ce que l'on pourrait potentiellement déduire de ces données :

    Advantage_gold : Si l'équipe bleue a un avantage financier significatif (c'est-à-dire, elle a accumulé plus de "gold" que l'équipe rouge), cela pourrait augmenter ses chances de gagner. L'or dans League of Legends est utilisé pour acheter des objets qui améliorent les capacités des champions, donc un avantage en or peut se traduire par un avantage en combat.

    Advantage_buildings : Si l'équipe bleue a détruit plus de bâtiments que l'équipe rouge, cela pourrait également augmenter ses chances de gagner. La destruction des bâtiments est un objectif clé dans League of Legends, car la destruction du bâtiment principal de l'équipe adverse (le "Nexus") est nécessaire pour gagner le match.

    Drakes : Les dragons sont des monstres neutres dans League of Legends qui, lorsqu'ils sont tués, donnent à l'équipe qui les a tués des bonus permanents pour le reste du match. Si une équipe a tué plus de dragons, cela pourrait augmenter ses chances de gagner.

    Herald : Le Rift Herald est un autre monstre neutre qui, lorsqu'il est tué, donne à l'équipe qui l'a tué un bonus temporaire qui peut aider à détruire les bâtiments de l'équipe adverse. Si une équipe a tué le Rift Herald, cela pourrait augmenter ses chances de gagner.

En résumé, ce code pourrait aider à comprendre comment différents facteurs de jeu contribuent à la victoire dans League of Legends. Les résultats pourraient être utiles pour les joueurs qui cherchent à améliorer leur stratégie de jeu, ou pour les développeurs du jeu qui cherchent à équilibrer les différents éléments du jeu.

5. A l’aide du dossier « champ » et de la colonne « matchId » disponible dans tous les fichiers CSV mis à
disposition, retrouvez les champions utilisés dans chaque équipe.
o Votre nouveau DataFrame devra être basé sur le fichier « match_full_time.csv »
o Il devra avoir deux colonnes ajoutées « blue_team » et « red_team » comportant respectivement les
champions (leur ID) dans chaque équipe, séparés par des virgules et encadrés par des [].
▪ Exemple : ["Aatrox","DrMundo","Katarina","Lux","Zoe"]
o Le DataFrame final devra être le même qu’initial, mais les deux colonnes doivent avoir été ajoutées
dans l’ordre entre les colonnes « tier » et « blue_kill ».
o Ce Dataframe devra être exporté au format JSON avec le nom « df_ex5.json »

In [22]:

import pandas as pd

# Lire les données du match
match_data = pd.read_csv("data/match_full_time.csv")
match_data['blue_team'] = [[] for _ in range(len(match_data))]
match_data['red_team'] = [[] for _ in range(len(match_data))]

# Liste de tous les noms des champions
champions = ["Aatrox", "Ahri", "Akali", "Alistar", "Amumu", "Anivia", "Annie", "Aphelios", "Ashe", "Aurelion Sol", 
             "Azir", "Bard", "Blitzcrank", "Brand", "Braum", "Caitlyn", "Camille", "Cassiopeia", "ChoGath", "Corki", 
             "Darius", "Diana", "Dr. Mundo", "Draven", "Ekko", "Elise", "Evelynn", "Ezreal", "Fiddlesticks", "Fiora", 
             "Fizz", "Galio", "Gangplank", "Garen", "Gnar", "Gragas", "Graves", "Hecarim", "Heimerdinger", "Illaoi", 
             "Irelia", "Ivern", "Janna", "Jarvan IV", "Jax", "Jayce", "Jhin", "Jinx", "KaiSa", "Kalista", "Karma", 
             "Karthus", "Kassadin", "Katarina", "Kayle", "Kayn", "Kennen", "KhaZix", "Kindred", "Kled", "KogMaw", 
             "LeBlanc", "Lee Sin", "Leona", "Lillia", "Lissandra", "Lucian", "Lulu", "Lux", "Malphite", "Malzahar", 
             "Maokai", "Master Yi", "Miss Fortune", "Mordekaiser", "Morgana", "Nami", "Nasus", "Nautilus", "Neeko", 
             "Nidalee", "Nocturne", "Nunu  Willump", "Olaf", "Orianna", "Ornn", "Pantheon", "Poppy", "Pyke", "Qiyana", 
             "Quinn", "Rakan", "Rammus", "RekSai", "Renekton", "Rengar", "Riven", "Rumble", "Ryze", "Sejuani", "Senna", 
             "Sett", "Shaco", "Shen", "Shyvana", "Singed", "Sion", "Sivir", "Skarner", "Sona", "Soraka", "Swain", "Sylas", 
             "Syndra", "Tahm Kench", "Taliyah", "Talon", "Taric", "Teemo", "Thresh", "Tristana", "Trundle", "Tryndamere", 
             "Twisted Fate", "Twitch", "Udyr", "Urgot", "Varus", "Vayne", "Veigar", "VelKoz", "Vi", "Viktor", "Vladimir", 
             "Volibear", "Warwick", "Wukong", "Xayah", "Xerath", "Xin Zhao", "Yasuo", "Yone", "Yorick", "Yuumi", "Zac", 
             "Zed", "Ziggs", "Zilean", "Zoe", "Zyra"]

# Parcourir tous les champions
for champ in champions:
    # Lire les données du champion
    champ_data = pd.read_csv(f"data/champ/{champ}.csv")
    
    # Fusionner les données du match et du champion sur matchId
    merged_data = pd.merge(match_data, champ_data, on='matchId')
    
    # Ajouter le champion à l'équipe correspondante pour chaque match
    for index, row in merged_data.iterrows():
        if row['blue_win'] == row['win']: # Si l'équipe bleue a gagné et que le champion a gagné, il est dans l'équipe bleue
            match_data.loc[match_data['matchId'] == row['matchId'], 'blue_team'].apply(lambda x: x.append(champ))
        else: # Sinon, le champion est dans l'équipe rouge
            match_data.loc[match_data['matchId'] == row['matchId'], 'red_team'].apply(lambda x: x.append(champ))

# Réorganiser les colonnes pour que 'blue_team' et 'red_team' soient avant 'blue_kill'
columns = ['matchId', 'makeTime', 'tier', 'blue_team', 'red_team', 'blue_kill', 'red_kill', 'blue_gold', 'red_gold', 'blue_tower', 'blue_inhibitor', 'red_tower', 'red_inhibitor', 'blue_monster', 'red_monster', 'blue_firstBlood', 'blue_firstTower', 'blue_firstInhibitor', 'blue_firstBaron', 'blue_firstDragon', 'blue_firstRiftHerald', 'gameDuration', 'blue_win']
match_data = match_data[columns]

# Exporter les données du match au format JSON
match_data.to_json("df_ex5.json", orient='records')

